In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

In [2]:
df_train = pd.read_csv('data/pair_sentences_train.csv',sep='|',index_col=0)
df_test = pd.read_csv('data/pair_sentences_train.csv',sep='|',index_col=0)

/Users/rubenstern/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df_train_pos = df_train[df_train['label']==1]
df_train_neg = df_train[df_train['label']==0].sample(n = df_train_pos.shape[0])

In [4]:
df_array = [df_train_pos,df_train_neg]
df_train_balanced = pd.concat(df_array)
df_train_balanced = df_train_balanced.sample(frac=1)

In [5]:
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
vectorizer.fit(df_train_balanced['sentence'].values.astype('U'))

vectorizer_previous = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
vectorizer_previous.fit(df_train_balanced['previous_sentence'].values.astype('U'))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:
X = vectorizer.transform(df_train_balanced['sentence'].values.astype('U'))
y = df_train_balanced['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [7]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

y_train_predict = lr.predict(X_train)

In [8]:
precision_recall_fscore_support(y_train, y_train_predict)

(array([ 0.83312956,  0.85780462]),
 array([ 0.86329913,  0.8266659 ]),
 array([ 0.84794608,  0.84194745]),
 array([104191, 103938]))

In [9]:
lr_coef = lr.coef_[0]
features_name = np.array(vectorizer.get_feature_names())

In [10]:
index_max = sorted(range(len(lr_coef)), key=lambda i: lr_coef[i])[-20:]
index_min = sorted(range(len(lr_coef)), key=lambda i: lr_coef[i])[:20]

In [11]:
lr_coef[index_max]

array([ 3.43550573,  3.52767672,  3.54246572,  3.55988741,  3.61788701,
        3.73835155,  3.74130251,  3.9467817 ,  3.97031784,  4.07926005,
        4.17505784,  4.69329527,  5.03918658,  5.52045031,  6.25434222,
        6.43157922,  7.40237599,  7.54818323,  8.51847091,  8.67922704])

In [12]:
lr_coef[index_min]

array([-4.42143188, -3.54174254, -3.30599776, -3.04862071, -3.02418918,
       -2.88853552, -2.73700232, -2.73447393, -2.71042666, -2.65827753,
       -2.5356599 , -2.52173247, -2.46290996, -2.43108327, -2.41208807,
       -2.38497627, -2.37671855, -2.35504861, -2.33944574, -2.33883983])

In [13]:
features_name[index_max]

array(['sexist', 'conspiracy', 'scrub', 'unbiased', 'yea', 'shitlord',
       'definitely', 'fault', 'amirite', 'gee', 'glad', 'dare', 'surely',
       'confirmed', 'good thing', 'sure', 'totally', 'yeah', 'clearly',
       'obviously'],
      dtype='<U2469')

In [14]:
features_name[index_min]

array(['itt', 'fuck yeah', 'reminds', 'feel like', 'kinda', 'honestly',
       'fucking', 'fuck', 'fucked', 'shit', 'irony', 'watched', 'florida',
       'lol', 'stupidity', 'lmfao', 'netflix', 'giveaway', 'eli5', 'toilet'],
      dtype='<U2469')